# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [10]:
!cat publications.tsv

category	pub_date	title	venue	excerpt	citation	url_slug	paper_url
fullpapers	2024-12-30	GoalTrack: Supporting Personalized Goal-Setting in Stroke Rehabilitation with Multimodal Activity Journaling	Proceedings of the ACM on Interactive, Mobile, Wearable and Ubiquitous Technologies (IMWUT)	Stroke is the leading cause of disability among adults, with motor impairments being the most significant complication. Stroke rehabilitation is critical for stroke survivors to regain independence in their daily activities. Central to this rehabilitation process is patient-centered goal-setting, a crucial philosophy underpinning personalized programs. However, mismatched expectations between stroke survivors and clinicians often lead to limited engagement from patients, which detracts from patient-centeredness. We envision that stroke survivors who engage in journaling activities can empower themselves to be more proactive, thereby enhancing the goal-setting process. To this end, we iteratively design

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [11]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,category,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,fullpapers,2024-12-30,GoalTrack: Supporting Personalized Goal-Settin...,"Proceedings of the ACM on Interactive, Mobile,...",Stroke is the leading cause of disability amon...,"Jong Ho Lee, Sunghoon Ivan Lee, and Eun Kyoung...",imwut-goaltrack,http://brac45.github.io/files/goaltrack-paper-...
1,fullpapers,2021-12-30,Understanding and Supporting Self-Tracking App...,"Proceedings of the ACM on Interactive, Mobile,...",People often face barriers to selecting self-t...,"Jong Ho Lee, Jessica Schroeder, and Daniel A. ...",imwut-selection,https://dl.acm.org/doi/10.1145/3494980
2,fullpapers,2020-12-17,Mapping and Taking Stock of Personal Informati...,"Proceedings of the ACM on Interactive, Mobile,...",The research community on the study and design...,"Daniel A. Epstein, Clara Caldeira, Mayara Cost...",imwut-mapping,https://doi.org/10.1145/3432231
3,fullpapers,2018-12-02,Neutralizing BLE Beacon-based Electronic Atten...,IEEE Access,Many emerging location- or proximity-based app...,"Moonbeom Kim, Jong Ho Lee, and Jeongyeup Paek....",ble-ieee-access,https://doi.org/10.1109/ACCESS.2018.2884488
4,shortpapers,2016-01-01,Immersive Gesture Interface Design for HMD Bas...,Extended Abstracts of HCI Korea 2016,Increased popularity of head-mounted displays ...,"Lee Yea Som, Wang Duk Seo, Jong Ho Lee, Bong-S...",ea-gesture-interfaces,https://www.dbpia.co.kr/pdf/pdfView?nodeId=NOD...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [14]:
import os

html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

publications = pd.read_csv("publications.tsv", sep="\t", header=0)


for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications""" + '"\n'

    md += """category: """ + item.category
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    #if len(str(item.slides_url)) > 5:
    #    md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    #if len(str(item.slides_url)) > 5:
        #md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

publications

,category,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,fullpapers,2024-12-30,GoalTrack: Supporting Personalized Goal-Settin...,"Proceedings of the ACM on Interactive, Mobile,...",Stroke is the leading cause of disability amon...,"Jong Ho Lee, Sunghoon Ivan Lee, and Eun Kyoung...",imwut-goaltrack,http://brac45.github.io/files/goaltrack-paper-...
1,fullpapers,2021-12-30,Understanding and Supporting Self-Tracking App...,"Proceedings of the ACM on Interactive, Mobile,...",People often face barriers to selecting self-t...,"Jong Ho Lee, Jessica Schroeder, and Daniel A. ...",imwut-selection,https://dl.acm.org/doi/10.1145/3494980
2,fullpapers,2020-12-17,Mapping and Taking Stock of Personal Informati...,"Proceedings of the ACM on Interactive, Mobile,...",The research community on the study and design...,"Daniel A. Epstein, Clara Caldeira, Mayara Cost...",imwut-mapping,https://doi.org/10.1145/3432231
3,fullpapers,2018-12-02,Neutralizing BLE Beacon-based Electronic Atten...,IEEE Access,Many emerging location- or proximity-based app...,"Moonbeom Kim, Jong Ho Lee, and Jeongyeup Paek....",ble-ieee-access,https://doi.org/10.1109/ACCESS.2018.2884488
4,shortpapers,2016-01-01,Immersive Gesture Interface Design for HMD Bas...,Extended Abstracts of HCI Korea 2016,Increased popularity of head-mounted displays ...,"Lee Yea Som, Wang Duk Seo, Jong Ho Lee, Bong-S...",ea-gesture-interfaces,https://www.dbpia.co.kr/pdf/pdfView?nodeId=NOD...


These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md  2018-12-02-ble-ieee-access.md
2010-10-01-paper-title-number-2.md  2020-12-17-imwut-mapping.md
2015-10-01-paper-title-number-3.md  2021-12-30-imwut-selection.md
2016-01-01-ea-gesture-interfaces.md 2024-12-30-imwut-goaltrack.md
